# Combining datasets in Pandas

From: K.A.

Combining datasets can provide many opportunities for research. Such joins can range from the simplest joins of two sets to more complex database joins that handle all possible set matches. 

First, we'll look at the Series and DataFrame objects using the pd.concat function. Then we'll explore merge and join with the pd.merge function.


Don't forget to import the necessary libraries

In [1]:
import pandas as pd
import numpy as np

Next, let's add a function that creates a DateFrame object of a specific form, which we will need next

In [2]:
def make_df(cols, ind):
  data = {c: [str(c) + str(i) for i in ind] for c in cols}           
  return pd.DataFrame(data, ind)
make_df('AVK', range(3))


,A,V,K
0,A0,V0,K0
1,A1,V1,K1
2,A2,V2,K2


Let's start with a simple concatenation using the pd.concat function.
This function can be used for simple concatenation of Series or DataFrame objects

In [3]:
ser1 = pd.Series(['A', 'V', 'K'], index=[1, 2, 3])       
ser2 = pd.Series(['R', 'G', 'B'], index=[4, 5, 6])       
pd.concat([ser1, ser2])


1    A
2    V
3    K
4    R
5    G
6    B
dtype: object

In [5]:
df1 = make_df('CD', [1, 2])       
df2 = make_df('CD', [3, 4])       
print("df1")
print(df1)
print("df2")
print(df2)
print("pd.concat([df1, df2]")
print(pd.concat([df1, df2]))

df1
    C   D
1  C1  D1
2  C2  D2
df2
    C   D
3  C3  D3
4  C4  D4
pd.concat([df1, df2]
    C   D
1  C1  D1
2  C2  D2
3  C3  D3
4  C4  D4


The concatenation takes place in the DataFrame object line by line

In [6]:
df3 = make_df('AB', [0, 1])       
df4 = make_df('CD', [0, 1])       
print('df3')
print(df3)
print('df4')
print(df4)
print('pd.concat([df3, df4], axis="columns")')
print(pd.concat([df3, df4], axis="columns"))  

df3
    A   B
0  A0  B0
1  A1  B1
df4
    C   D
0  C0  D0
1  C1  D1
pd.concat([df3, df4], axis="columns")
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1


We could have specified axis=1, but we used the more intuitive option axis="columns".

It is important to remember that the concatenation from Pandas preserves the indexes, even if they are duplicated

In [7]:
x = make_df('AV', [0, 1])       
y = make_df('AV', [2, 3])       
y.index = x.index  # Duplicate indexes      
print("x")
print(x)
print("y")
print(y)
print("pd.concat([x, y])") 
print(pd.concat([x, y])) 

x
    A   V
0  A0  V0
1  A1  V1
y
    A   V
0  A2  V2
1  A3  V3
pd.concat([x, y])
    A   V
0  A0  V0
1  A1  V1
0  A2  V2
1  A3  V3


Note the repeating indices, but sometimes this result can be incorrect. So there are several ways to solve this problem.

One way to solve it is to specify ***verify_integrity***, which will output an error when the indexes are repeated. 

In [8]:
try:            
  pd.concat([x, y], verify_integrity=True)        
except ValueError as e:            
  print("ValueError:", e)


ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


Another way to solve the index duplication problem is to ignore it.

Sometimes the index does not matter and can be ignored by specifying ignore_index. If it is True, a new integer index will be created for the final result. 

In [9]:
print("x")
print(x)
print("y")
print(y)
print("pd.concat([x, y], ignore_index=True)")
print(pd.concat([x, y], ignore_index=True))

x
    A   V
0  A0  V0
1  A1  V1
y
    A   V
0  A2  V2
1  A3  V3
pd.concat([x, y], ignore_index=True)
    A   V
0  A0  V0
1  A1  V1
2  A2  V2
3  A3  V3


Another option is to add multi-index keys.  The result will be hierarchically indexed rows

In [10]:
print("x")
print(x)
print("y")
print(y)
print("pd.concat([x, y], keys=['x', 'y'])")
print(pd.concat([x, y], keys=['x', 'y']))


x
    A   V
0  A0  V0
1  A1  V1
y
    A   V
0  A2  V2
1  A3  V3
pd.concat([x, y], keys=['x', 'y'])
      A   V
x 0  A0  V0
  1  A1  V1
y 0  A2  V2
  1  A3  V3


The result is a multi-indexed DataFrame object

### Concatenation using joins.

In practice, data from different sources may have different sets of column names. The pd.concat() function has several options for this case. 

In [12]:
df5 = make_df('ABC', [1, 2])        
df6 = make_df('BCD', [3, 4])        
print("df5")
print(df5)
print("df6")
print(df6)
print("pd.concat([df5, df6])")
print(pd.concat([df5, df6]))

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2
df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4


By default, elements for which there is no data are filled with NA values. To change this, you can specify a value for the *join* option. By default the join is a union of the input columns, but it is possible to change this to an intersection of columns using **join='inner'**

In [14]:
print("df5")
print(df5)
print("df6")
print(df6)
print("pd.concat([df5, df6], join='inner')")
print(pd.concat([df5, df6], join='inner'))


df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2
df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
pd.concat([df5, df6], join='inner')
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4


Next, look at the append() method, which allows you to do the same thing, but with less effort

In [15]:
print("df1")
print(df1)
print("df2")
print(df2)
print("df1.append(df2)")
print(df1.append(df2))


df1
    C   D
1  C1  D1
2  C2  D2
df2
    C   D
3  C3  D3
4  C4  D4
df1.append(df2)
    C   D
1  C1  D1
2  C2  D2
3  C3  D3
4  C4  D4


Unlike append() and extend() in Python, the append() method in Pandas does not change the original object, but creates a new object with merged data. This makes this method not very efficient because it creates a new index.

Let's look at the join method with pd.merge(). In this method, there are three types of one-to-one, many-to-one, and many-to-many join and the same pd.merge() call.

Let's start with one-to-one. This is the simplest way to merge, much like column concatenation.

In [16]:
df1 = pd.DataFrame({'id':['phone','pencil','pen','bottle','eraser'],                    
                    'price': [12.33,11.44,33.21,13.23,33.62]})
df2 = pd.DataFrame({'id':['pencil','bottle','phone','pen','eraser'],                    
                    'color': ['white','red','pink','black','red']}) 
print("df1")
print(df1)
print("df2")
print(df2)

df1
       id  price
0   phone  12.33
1  pencil  11.44
2     pen  33.21
3  bottle  13.23
4  eraser  33.62
df2
       id  color
0  pencil  white
1  bottle    red
2   phone   pink
3     pen  black
4  eraser    red


To merge these objects into one, use  pd.merge()

In [17]:
df3 = pd.merge(df1, df2)       
print("df3")
print(df3)

df3
       id  price  color
0   phone  12.33   pink
1  pencil  11.44  white
2     pen  33.21  black
3  bottle  13.23    red
4  eraser  33.62    red


The pd.merge() function recognizes that both DataFrame objects have an id column, and automatically performs the merge using that column as the key. The merge results in a new object that combines information from the two input objects.

Now consider the "many-to-one" join method. In this method, one of the two columns contains repeating values.


In [18]:
df4 = pd.DataFrame({'color': ['white','red','pink','black'],                           
                     'brand': ['MLO','ABK','AVK','PDF']})       
print("df3")
print(df3)
print("df4")
print(df4)
print("pd.merge(df3, df4)")
print(pd.merge(df3, df4))

df3
       id  price  color
0   phone  12.33   pink
1  pencil  11.44  white
2     pen  33.21  black
3  bottle  13.23    red
4  eraser  33.62    red
df4
   color brand
0  white   MLO
1    red   ABK
2   pink   AVK
3  black   PDF
pd.merge(df3, df4)
       id  price  color brand
0   phone  12.33   pink   AVK
1  pencil  11.44  white   MLO
2     pen  33.21  black   PDF
3  bottle  13.23    red   ABK
4  eraser  33.62    red   ABK


The resulting DataFrame object has an additional column with brand information, with information repeated in one or more places according to the data entered.

Next, consider the "many-to-many" joining method.

In [19]:
df5 = pd.DataFrame({'color': ['white','red','red','pink','black','black'],                           
                     'skills': ['math', 'ML', 'coding','AI','ML', 'Python']}) 
print("df1")
print(df1)
print("df5")
print(df5)
print("pd.merge(df1, df5)")
print(pd.merge(df2, df5))


df1
       id  price
0   phone  12.33
1  pencil  11.44
2     pen  33.21
3  bottle  13.23
4  eraser  33.62
df5
   color  skills
0  white    math
1    red      ML
2    red  coding
3   pink      AI
4  black      ML
5  black  Python
pd.merge(df1, df5)
       id  color  skills
0  pencil  white    math
1  bottle    red      ML
2  bottle    red  coding
3  eraser    red      ML
4  eraser    red  coding
5   phone   pink      AI
6     pen  black      ML
7     pen  black  Python


Now let's look at the way by setting the merge key. The names of the columns do not match exactly, so there are parameters for this situation.

One of these parameters is the keyword "on". This is one of the simplest ways, in which you specify a name or a list of column names

In [20]:
print("df1")
print(df1)
print("df2")
print(df2)
print("pd.merge(df1, df2, on='id')")
print(pd.merge(df1, df2, on='id'))

df1
       id  price
0   phone  12.33
1  pencil  11.44
2     pen  33.21
3  bottle  13.23
4  eraser  33.62
df2
       id  color
0  pencil  white
1  bottle    red
2   phone   pink
3     pen  black
4  eraser    red
pd.merge(df1, df2, on='id')
       id  price  color
0   phone  12.33   pink
1  pencil  11.44  white
2     pen  33.21  black
3  bottle  13.23    red
4  eraser  33.62    red


This parameter works only when the left and right DataFrame objects have the specified column name.

The next parameter we will consider is the "left_on" and "right_on" parameters. This method is convenient when you have to merge two datasets with different column names. In this case, you can use the keywords "left_on" and "right_on" to specify the names of the two columns you want

In [21]:
df3 = pd.DataFrame({'title':['phone','pencil','pen','bottle','eraser'],
                    'commission': [7, 8, 6, 9,5]}) 
print("df1")
print(df1)
print("df3")
print(df3)
print("pd.merge(df1, df3, left_on='id', right_on='title')")
print(pd.merge(df1, df3, left_on="id", right_on="title"))

df1
       id  price
0   phone  12.33
1  pencil  11.44
2     pen  33.21
3  bottle  13.23
4  eraser  33.62
df3
    title  commission
0   phone           7
1  pencil           8
2     pen           6
3  bottle           9
4  eraser           5
pd.merge(df1, df3, left_on='id', right_on='title')
       id  price   title  commission
0   phone  12.33   phone           7
1  pencil  11.44  pencil           8
2     pen  33.21     pen           6
3  bottle  13.23  bottle           9
4  eraser  33.62  eraser           5


The result of this action contains a redundant column that can be deleted if desired. For example, using the drop() method available in DataFrame objects

In [22]:
pd.merge(df1, df3, left_on="id", right_on="title").drop('title', axis=1)

,id,price,commission
0,phone,12.33,7
1,pencil,11.44,8
2,pen,33.21,6
3,bottle,13.23,9
4,eraser,33.62,5


The parameters left_index and right_index are convenient for index merging

In [23]:
df1a = df1.set_index('id')       
df2a = df2.set_index('id')       
print("df1a")
print(df1a)
print("df2a")
print(df2a)

df1a
        price
id           
phone   12.33
pencil  11.44
pen     33.21
bottle  13.23
eraser  33.62
df2a
        color
id           
pencil  white
bottle    red
phone    pink
pen     black
eraser    red


You can use an index as a merge key by specifying the flags left_index and/or right_index in the pd.merge() method

In [24]:
print('df1a')
print(df1a)
print('df2a')
print(df2a)
print('pd.merge(df1a, df2a, left_index=True, right_index=True)')
print(pd.merge(df1a, df2a, left_index=True, right_index=True))

df1a
        price
id           
phone   12.33
pencil  11.44
pen     33.21
bottle  13.23
eraser  33.62
df2a
        color
id           
pencil  white
bottle    red
phone    pink
pen     black
eraser    red
pd.merge(df1a, df2a, left_index=True, right_index=True)
        price  color
id                  
phone   12.33   pink
pencil  11.44  white
pen     33.21  black
bottle  13.23    red
eraser  33.62    red


There is a join() method in DataFrame objects that performs index merge by default

In [25]:
print('df1a')
print(df1a)
print('df2a')
print(df2a)
print('df1a.join(df2a)')
print(df1a.join(df2a))

df1a
        price
id           
phone   12.33
pencil  11.44
pen     33.21
bottle  13.23
eraser  33.62
df2a
        color
id           
pencil  white
bottle    red
phone    pink
pen     black
eraser    red
df1a.join(df2a)
        price  color
id                  
phone   12.33   pink
pencil  11.44  white
pen     33.21  black
bottle  13.23    red
eraser  33.62    red


If a combination of column and index merges is required, then use a combination of the "left_index" flag with the "right_on" parameter or the "left_on" parameter with the "right_index" flag

In [26]:
print("df1a")
print(df1a)
print("df3")
print(df3)
print("pd.merge(df1a, df3, left_index=True, right_on='title')")
print(pd.merge(df1a, df3, left_index=True, right_on='title'))

df1a
        price
id           
phone   12.33
pencil  11.44
pen     33.21
bottle  13.23
eraser  33.62
df3
    title  commission
0   phone           7
1  pencil           8
2     pen           6
3  bottle           9
4  eraser           5
pd.merge(df1a, df3, left_index=True, right_on='title')
   price   title  commission
0  12.33   phone           7
1  11.44  pencil           8
2  33.21     pen           6
3  13.23  bottle           9
4  33.62  eraser           5


Let's consider the case when some value is in one key column, but not in the other

In [30]:
df6 = pd.DataFrame({'name': ['John', 'Katie', 'Linda'],
                     'food': ['water', 'potato', 'onion']},
                      columns=['name', 'food'])        
df7 = pd.DataFrame({'name': ['Linda', 'Duo'],
                     'drink': ['juice', 'milk']},
                      columns=['name', 'drink'])       
print("df6")
print(df6)
print("df7")
print(df7)
print("pd.merge(df6, df7)")
print(pd.merge(df6, df7))

df6
    name    food
0   John   water
1  Katie  potato
2  Linda   onion
df7
    name  drink
0  Linda  juice
1    Duo   milk
pd.merge(df6, df7)
    name   food  drink
0  Linda  onion  juice


Here we merged two datasets with only one name record matching: Linda. By default, the result will contain the intersection of the two input sets - an inner join. You can specify this explicitly with the how keyword, which has the default value 'inner'

In [31]:
pd.merge(df6, df7, how='inner')


,name,food,drink
0,Linda,onion,juice


Other possible meanings of the keyword 'how' are 'outer', 'left' and 'right'. "Outer join" means join by union of input columns and fills with NA values all missing values

In [32]:
print("df6")
print(df6)
print("df7")
print(df7)
print("pd.merge(df6, df7, how='outer')")
print(pd.merge(df6, df7, how='outer'))

df6
    name    food
0   John   water
1  Katie  potato
2  Linda   onion
df7
    name  drink
0  Linda  juice
1    Duo   milk
pd.merge(df6, df7, how='outer')
    name    food  drink
0   John   water    NaN
1  Katie  potato    NaN
2  Linda   onion  juice
3    Duo     NaN   milk


The left join and the right join perform the joining of the left and right values, respectively

In [33]:
print("df6")
print(df6)
print("df7")
print(df7)
print("pd.merge(df6, df7, how='left')")
print(pd.merge(df6, df7, how='left'))

df6
    name    food
0   John   water
1  Katie  potato
2  Linda   onion
df7
    name  drink
0  Linda  juice
1    Duo   milk
pd.merge(df6, df7, how='left')
    name    food  drink
0   John   water    NaN
1  Katie  potato    NaN
2  Linda   onion  juice


The result lines now correspond to the entries in the left of the input objects. The option how='right' works the same way. 

Sometimes there may be a case where two input objects have conflicting column names.


In [34]:
df8 = pd.DataFrame({'name': ['Lucy', 'Ann', 'Steven', 'Alice'],
                     'rank': [1, 2, 3, 4]})        
df9 = pd.DataFrame({'name': ['Lucy', 'Ann', 'Steven', 'Alice'],
                     'rank': [4, 1, 3, 2]})        
print("df8")
print(df8)
print("df9")
print(df9)
print('pd.merge(df8, df9, on="name")')
print(pd.merge(df8, df9, on="name"))

df8
     name  rank
0    Lucy     1
1     Ann     2
2  Steven     3
3   Alice     4
df9
     name  rank
0    Lucy     4
1     Ann     1
2  Steven     3
3   Alice     2
pd.merge(df8, df9, on="name")
     name  rank_x  rank_y
0    Lucy       1       4
1     Ann       2       1
2  Steven       3       3
3   Alice       4       2


Since the result should have two conflicting column names, the merge function automatically added the suffixes _x and _y to the names to ensure that the column names of the result are unique. 

You can also specify other suffixes using the "suffixes" keyword

In [35]:
print('df8')
print(df8)
print('df9')
print(df9)
print('pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')
print(pd.merge(df8, df9, on="name", suffixes=["_L", "_R"]))

df8
     name  rank
0    Lucy     1
1     Ann     2
2  Steven     3
3   Alice     4
df9
     name  rank
0    Lucy     4
1     Ann     1
2  Steven     3
3   Alice     2
pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
     name  rank_L  rank_R
0    Lucy       1       4
1     Ann       2       1
2  Steven       3       3
3   Alice       4       2


These suffixes will work for all possible connections, including multiple overlapping column names.